<a href="https://colab.research.google.com/github/Banking-Analytics-Lab/DLinBankingBook/blob/main/Labs/TextBook_Lab_Chap6_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 6 Lab - Large Language Models

In this lab, we will explore prompt engineering and fine-tuning for large language models (LLMs) using LLaMA 3.2 (3B) - Instruct.

We will start by installing and importing the required libraries.

In [ ]:
!pip install -U transformers accelerate datasets peft shap
!pip install -U xgboost==3.0.5
!pip install -q -U bitsandbytes==0.45.2 trl fsspec==2025.3.2
!pip install --no-build-isolation https://github.com/Dao-AILab/flash-attention/releases/download/v2.8.3/flash_attn-2.8.3+cu12torch2.8cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

In [ ]:
# Imports
import numpy as np
import os
import pandas as pd

# Set environmental variables
from google.colab import userdata
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "300"  # Set timeout to 5 minutes
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from IPython.display import Markdown, display
import graphviz
from matplotlib.colors import ListedColormap
graphviz.set_jupyter_format('png')
%matplotlib inline

# Import Pytorch lybraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.optim.lr_scheduler import _LRScheduler

# Huggingface
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import pipeline, logging, HfArgumentParser
from transformers import set_seed
import accelerate
from datasets import load_dataset

# XGBoost
from xgboost import XGBClassifier

# Shap
import shap

# peft
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

# wandb
import wandb

# trl
from trl import SFTTrainer, clone_chat_template

#Bitsandbytes
import bitsandbytes as bnb


## Basic prompt engineering

To optimize model execution, we use Hugging Face's Accelerate library, which helps efficiently manage device placement (CPU/GPU).

In [ ]:
# Add accelerator
accelerator = accelerate.Accelerator()
device = accelerator.device

Before accessing restricted models or using private repositories, we need to authenticate with Hugging Face Hub.

The following command prompts for your Hugging Face access token to enable secure model downloads and API access:

How to Get Your Token?

Visit Hugging Face Token Page.
Generate a new token (select "Write" permissions if needed).
Copy and paste the token when prompted in Colab.
Once logged in, you can download gated models, access private datasets, and use Hugging Face services seamlessly.

In [ ]:
login(token=os.environ["HF_TOKEN"])

In this step, we load the LLaMA 3.2 (3B)-Instruct model using Hugging Face's transformers library and configure it for optimized execution on Google Colab's GPU.

* The model we are using is LLaMA 3.2 (3B) Instruct, a fine-tuned version designed for instruction-following tasks.
* We use [bfloat16](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format) (Brain Floating Point 16-bit), a precision format that improves memory efficiency while maintaining accuracy.

Downloading the model will take approximately 3 minutes.

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, dtype=dtype, device_map="auto")
model = accelerator.prepare(model)  # Ensure model is placed on the correct device

The following code displays the chat template used by the LLaMA 3.2 (3B) Instruct model to format conversations.

What is a Chat Template?

A chat template defines the structure of input messages before they are passed to the model. It ensures that the model correctly interprets the conversation flow and generates appropriate responses.

If a chat template is not available, prompts must be manually formatted.
In our case, we have a predefined chat template, simplifying the process.

In [ ]:
print(tokenizer.chat_template)

This code ensures that both the tokenizer and model configuration have a defined padding token to handle variable-length inputs properly.

In [ ]:
# Ensure tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

Now, we initialize a text generation pipeline using the Hugging Face **`transformers`** library.

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    dtype=dtype,
    device_map="auto",
)

Then, we format a conversation into a structured prompt using a chat template for the model.

* The system message sets the model’s behavior.
* The user message provides the specific query.

In [ ]:
# Example Input
system_message = "You are an expert assistant specializing in Banking Analytics and Business Analytics. Provide structured and factual responses."
user_prompt = "Explain the key differences between Banking Analytics and Business Analytics"

messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "user",
        "content": user_prompt,
    },
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

The following code generates text using a pre-trained language model via the pipe (text generation pipeline) and prints the generated response.


* max_new_tokens=512 → Limits the response length to 512 tokens.
* do_sample=True → Enables sampling (randomness) instead of deterministic outputs.
* top_p=0.8 → Uses nucleus sampling, meaning the model selects from the top 80% probability mass.
* temperature=0.2 → Controls randomness:

    Lower values (e.g., 0.2) → More focused and  deterministic responses.

    Higher values (e.g., 1.0) → More creative and diverse responses.

* repetition_penalty=1.5 → Penalizes repeated phrases to make the response more natural and varied.

In [ ]:
# Set seed for reproducibility
set_seed(42)

outputs = pipe(prompt, max_new_tokens=512,
               do_sample=True,
               top_p=0.8,
               temperature=0.2,
               repetition_penalty=1.5)
print(outputs[0]["generated_text"])

It's working well! Let's improve readability.

In [ ]:
display(
    Markdown(
            outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>"
            )[1]
        )
    )

The response is well-written, but with prompt engineering, we can refine it further for improved structure and formatting.

Now, let's enhance it by providing more specific instructions for the response.

In [ ]:
# Example Input
system_message = """STRICT INSTRUCTIONS:
1. First, provide a **clear definition** of the topic.
2. Then, explain **at least two key differences** in a **structured manner**.
   - Each difference must be in a **separate paragraph**.
   - Use **clear and concise language**.
3. Do not include unnecessary information or extra details beyond the requested explanation.
"""


user_prompt = "What is Banking Analytics, and how does it differ from general Business Analytics?"


# Use chat template for proper formatting
messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "user",
        "content": user_prompt,
    },
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [ ]:
# Set seed for reproducibility
set_seed(42)

outputs = pipe(prompt, max_new_tokens=512,
               do_sample=True,
               top_p=0.8,
               temperature=0.2,
               repetition_penalty=1.5)

display(
    Markdown(
            outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>"
            )[1]
        )
    )

The model's response is now more structured and readable, accurately following the instructions provided.

## Prompt engineering with prediction model

In this section, we will use prompt engineering to guide the LLM in explaining delinquency status predictions based on SHAP values.

Let's begin by downloading the dataset to proceed with our analysis.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1nrhxfnAkI0bZRXJiWu_JVKusAD9iHBpK/view?usp=sharing'

In [ ]:
df = pd.read_csv('loan_app.csv')
df

Next, we will prepare the dataset for training by applying feature encoding, scaling, and train-test splitting to ensure the model learns effectively from the data.

In [ ]:
X = df.drop(columns=["target"])  # Features
y = df["target"]  # Target variable

# Convert Categorical Features to Numerical
categorical_columns = X.select_dtypes(include=["object"]).columns.tolist()
numerical_columns = X.select_dtypes(exclude=["object"]).columns.tolist()

# Apply One-Hot Encoding
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_encoded = encoder.fit_transform(X[categorical_columns])

# Convert Encoded Data to DataFrame
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Drop original categorical columns and merge one-hot encoded features
X = X.drop(columns=categorical_columns)
X = pd.concat([X, X_encoded_df], axis=1)

# Train-Test Split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale Only Numerical Features (NOT one-hot encoded features)
scaler = StandardScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])  # Use same scaler for test set

X_train

Now, we will train an XGBoost model to predict loan delinquency status.

In [ ]:
negative_count = np.sum(y_train == 0)  # Count of class 0
positive_count = np.sum(y_train == 1)  # Count of class 1
scale_pos_weight = negative_count / positive_count  # Weight ratio

# Train XGBoost Classifier
xgb_model = XGBClassifier(max_depth=3,
                          learning_rate=0.01,
                          n_estimators=200,
                          verbosity=0,
                          objective='binary:logistic',
                          eval_metric="logloss",
                          booster='gbtree',
                          n_jobs=-1,
                          gamma=0.001,
                          subsample=0.632,
                          colsample_bytree=1,
                          colsample_bylevel=1,
                          colsample_bynode=1,
                          reg_alpha=0,
                          reg_lambda=0.1,
                          random_state=428,
                          tree_method="hist",
                          scale_pos_weight=scale_pos_weight
                          )

xgb_model.fit(X_train, y_train)

# Make Predictions
y_pred = xgb_model.predict(X_test)

# Evaluate Model Performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

We analyzes feature importance in the XGBoost model using SHAP (SHapley Additive exPlanations) to explain why the model predicts a loan as delinquent or not.

The SHAP summary plot provides:
* Feature importance ranking (sorted by impact).
* How each feature affects predictions (positive/negative impact).
* Distribution of SHAP values for different feature values.

In [ ]:
# Explain Model Predictions using SHAP
explainer = shap.TreeExplainer(xgb_model, X_train)
shap_values = explainer(X_test)  # SHAP values for test set

# Summarize SHAP Values
shap.summary_plot(shap_values, X_test)

We retrieve and format SHAP values for one test sample, making it easier to generate human-readable explanations using Llama.

In [ ]:
# Select an example loan case (e.g., first sample in test set)
sample_index = 10  # Change this index if needed
shap_values_sample = shap_values[sample_index].values
sample_features = X_test.iloc[sample_index]

# Convert SHAP values into dictionary format for input
shap_dict = {feature: shap_values_sample[i] for i, feature in enumerate(sample_features.index)}

# Target label for the selected sample
pred_label = y_pred[sample_index]

print("\nSHAP Values for Sample Client:\n", shap_dict)

Before proceeding to Llma, let's generate a bar plot of SHAP values for the sampled loan to visualize which features have the most impact on the delinquency prediction.

In [ ]:
shap.plots.bar(shap_values[sample_index])

Finally, we format SHAP values into a structured prompt to enable Llama to generate an interpretable explanation of why a loan is predicted as delinquent or not.

We first sort SHAP values by absolute magnitude, prioritizing the most impactful features. Then, we select the top 3 features to keep the explanation concise and focused.

In the prompt, we will:

* Ensure a structured format for clear and consistent LLaMA responses.
* Eliminate unnecessary or unrelated information from the generated output.
* Clearly state the model's decision using SHAP-based reasoning.

In [ ]:
def format_shap_explanation(system_message, sample_features, shap_dict, pred_label, scaler):
    # Convert scaled numerical values back to original values
    original_values = scaler.inverse_transform(sample_features[numerical_columns].values.reshape(1, -1))
    original_feature_values = {feature: original_values[0][i] for i, feature in enumerate(numerical_columns)}

    # Sort SHAP values by absolute magnitude (most impactful features first)
    top_features = sorted(shap_dict.items(), key=lambda x: abs(x[1]), reverse=True)[:3]

    # Generate SHAP explanation text
    shap_text = "\n".join(
        [f"{feature}: SHAP value = {shap_value:.4f}, feature value = {original_feature_values[feature]:.2f}"
         for feature, shap_value in top_features]
    )

    print(shap_text)
    # Define loan delinquency status
    delinquency_status = "likely to be delinquent" if pred_label == 1 else "unlikely to be delinquent"
    print("predicted delinquency: ", delinquency_status)

    # Construct a revised prompt with explicit instructions and structured format
    user_prompt = f"""
The model predicts that the client is {delinquency_status}.

Here are the three most important features influencing the prediction:

{shap_text}

### Instructions:
- Analyze how each of three features contributes to the prediction.
- **Use correct feature names, not feature values.**
- **Strictly follow the structured response format.**
- **SHAP values must be interpreted correctly**:
  - **A positive SHAP value means the feature increases delinquency risk.**
  - **A negative SHAP value means the feature decreases delinquency risk.**

### Response Format:
Feature Name: [Feature Name]
Effect on Risk: Explain whether a higher or lower value increases delinquency risk and why.
SHAP Impact: Clearly state whether the SHAP value shows an increase or decrease in delinquency risk and explain its significance.

### Example Response:

Feature Name: Credit Score
Effect on Risk: A higher credit score reduces delinquency risk because it indicates a strong repayment history and financial responsibility.
SHAP Impact: The SHAP value **-0.5862** shows that **including credit score for this client decreases** the probability of delinquency, meaning the model considers this a strong indicator of financial reliability.

Now begin your structured analysis:
"""


    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt


We also define instructions to guide the LLM in analyzing SHAP values for loan delinquency predictions.

In [ ]:
system_message = """
You are a financial risk analyst. Your job is to analyze SHAP values and provide structured, fact-based explanations of the model's predictions.

Guidelines:
1. Interpret SHAP values correctly:
   - **A positive SHAP value means the feature increases delinquency risk.**
   - **A negative SHAP value means the feature decreases delinquency risk.**
2. **Do not contradict basic financial logic**:
   - A **higher credit score should always reduce risk** unless explicitly stated otherwise.
3. **Strictly follow the response format. Do not add extra text or repeat information.**
4. **Do not argue against the given ranking of features.**
5. **Avoid repetition, unnecessary details, or ranking errors.**
"""



prompt = format_shap_explanation(system_message, sample_features, shap_dict, pred_label, scaler)

Finally, let's give it a try!

In [ ]:
# Set seed for reproducibility
set_seed(42)

outputs = pipe(prompt, max_new_tokens=512,
               do_sample=True,
               top_p=0.5,
               temperature=0.1,
               repetition_penalty=1)

In [ ]:
display(
    Markdown(
            outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>"
            )[1]
        )
    )

The model provides a fairly good interpretation of SHAP values, but the quality can be improved. We can enhance its performance through fine-tuning.

## Fine-tuning Llama 3.2 - 3B Instruct

In this section, we will fine-tune the LLaMA 3.2 - 3B Instruct model to enhance its ability to interpret SHAP values for loan delinquency predictions. Our goal is to tailor the model to generate clear, insightful explanations that financial institutions can use for credit scoring and risk assessment.



The following code initializes Weights & Biases (W&B) for experiment tracking and logging during the fine-tuning process. A W&B authentication token is required to enable logging.

To use W&B properly, ensure that you have set up an account and retrieved your authentication token from W&B before running wandb.login().

In [ ]:
wandb.login()
run = wandb.init(
    project='Fine-tune Llama 3.2',
    job_type="training",
    anonymous="allow"
)

To proceed with fine-tuning, we need to download the dataset that will be used for training the model.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/15JtT_Jw9OdS3ZvecrYY8gB0s9jiyvEEa/view?usp=sharing'

We define the base model (Llama 3.2 3B Instruct), the name for the fine-tuned model, and the dataset used for training.

In [ ]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"
new_model = "llama-3.2-3b-it-SHAP-Explainer"
dataset_name = "llama3.2_finetune_data.json"

The following Checks the GPU's compute capability using **`torch.cuda.get_device_capability()`**.

If the GPU supports compute capability 8.0 or higher (e.g., A100 on Colab Pro), it enables Flash Attention (flash_attention_2) for optimized training.
Otherwise, it defaults to the standard ("eager") attention mechanism, ensuring compatibility with older GPUs.

In [ ]:
# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"

else:
    torch_dtype = torch.bfloat16
    attn_implementation = "eager"

print(f"Using {attn_implementation} for training.")

We set up and load a quantized LLaMA model using QLoRA (Quantized Low-Rank Adaptation) for memory-efficient fine-tuning with 4-bit precision.

In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")

# Ensure tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id

# Explicitly cast lm_head weight to the compute dtype
if hasattr(model, 'lm_head'):
    model.lm_head.weight.data = model.lm_head.weight.data.to(torch_dtype)

We will load and prepare the fine-tuning dataset from a local JSON file containing 1,000 samples.

It is then split into training (90%) and testing (10%) for evaluation.

In [ ]:
# Load the local JSON dataset using pandas
df_dataset = pd.read_json("llama3.2_finetune_data.json")

# Convert the pandas DataFrame to a Hugging Face Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df_dataset)

dataset = dataset.train_test_split(test_size=0.1, seed=42)

Our dataset is structured as follows:

In [ ]:
# Preview the dataset
print(dataset['train'][0])

We will define the following system message to guide the model during fine-tuning.

In [ ]:
system_message = """
You are a financial risk analyst. Your job is to analyze SHAP values and provide structured, fact-based explanations of the model's predictions.
"""

We shuffle and format the dataset to align with the chat-based fine-tuning format.

In [ ]:
dataset = dataset.shuffle(seed=42)

def format_chat_template(row):

    row_json = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": row['messages'][1]['content']},
       {"role": "assistant", "content": row['messages'][2]['content']+" <|eot_id|>"}
    ]

    #row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    row["text"] = tokenizer.apply_chat_template(conversation=row_json, tokenize=False)

    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= 4,
)

The following identifies all linear layers in the model that are quantized using BitsAndBytes (bnb) 4-bit precision and returns a list of their names. These module names are typically used when applying LoRA (Low-Rank Adaptation).

In [ ]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

Now, we configure and apply LoRA to the model, optimizing it for efficient fine-tuning with reduced memory usage.

* `r=16`: Rank of LoRA adaptation (controls memory usage vs. expressiveness).
* `lora_alpha=32`: Scaling factor for LoRA layers.
* `lora_dropout=0.05`: Dropout rate to prevent overfitting.
* `bias="none"`: Ensures no extra biases are added to the model.
* `target_modules=modules`: LoRA is applied only to specific layers identified earlier.

In [ ]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

# Reset chat template before reapplying it
tokenizer.chat_template = None

model, tokenizer, added_tokens = clone_chat_template(model, tokenizer, base_model)
model = get_peft_model(model, peft_config)

Now, let's defines the training hyperparameters for fine-tuning the model using the Hugging Face TrainingArguments class.


* `Batch Sizes`: Processes one sample per GPU for both training and evaluation.
* `Gradient Accumulation`: Accumulates gradients over one step before updating weights to reduce memory usage.
* `Optimizer`: Uses Paged AdamW 32-bit, optimized for memory efficiency.
* `Evaluation Strategy`: Evaluates every 20% of an epoch.
* `Learning Rate & Warmup`: Starts with a 0.0002 learning rate and warms up for 10 steps.
* `Precision`: Uses bfloat16 (BF16) for faster training and reduced memory consumption.

In [ ]:
#Hyperparamter tuning. Batch size of 2 fits in 21.4 GB of VRAM. Increase if you have more.
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    bf16=True,
    report_to="wandb",
    seed=42,
)

We initializes the SFTTrainer for supervised fine-tuning.

In [ ]:
set_seed(42)

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_arguments,
    processing_class=tokenizer
)

Everything is set up! Let's begin training! It takes approximately 7 minutes on an A100 GPU.

In [ ]:
# takes around 10 min with L4
set_seed(42)
trainer.train()

After training, let's properly close the Weights & Biases (wandb) logging session.

In [ ]:
wandb.finish()

Now, let's test our fine-tuned model using the same dataset from before.

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1nrhxfnAkI0bZRXJiWu_JVKusAD9iHBpK/view?usp=sharing'

In [ ]:
df = pd.read_csv('loan_app.csv')

In [ ]:
X = df.drop(columns=["target"])  # Features
y = df["target"]  # Target variable

# Convert Categorical Features to Numerical
categorical_columns = X.select_dtypes(include=["object"]).columns.tolist()
numerical_columns = X.select_dtypes(exclude=["object"]).columns.tolist()

# Apply One-Hot Encoding
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_encoded = encoder.fit_transform(X[categorical_columns])

# Convert Encoded Data to DataFrame
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Drop original categorical columns and merge one-hot encoded features
X = X.drop(columns=categorical_columns)
X = pd.concat([X, X_encoded_df], axis=1)

# Train-Test Split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale Only Numerical Features (NOT one-hot encoded features)
scaler = StandardScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])  # Use same scaler for test set

In [ ]:
negative_count = np.sum(y_train == 0)  # Count of class 0
positive_count = np.sum(y_train == 1)  # Count of class 1
scale_pos_weight = negative_count / positive_count  # Weight ratio

# Train XGBoost Classifier
xgb_model = XGBClassifier(max_depth=3,
                          learning_rate=0.01,
                          n_estimators=200,
                          verbosity=0,
                          objective='binary:logistic',
                          eval_metric="logloss",
                          booster='gbtree',
                          n_jobs=-1,
                          gamma=0.001,
                          subsample=0.632,
                          colsample_bytree=1,
                          colsample_bylevel=1,
                          colsample_bynode=1,
                          reg_alpha=0,
                          reg_lambda=0.1,
                          random_state=428,
                          tree_method="hist",
                          scale_pos_weight=scale_pos_weight
                          )

xgb_model.fit(X_train, y_train)

# Make Predictions
y_pred = xgb_model.predict(X_test)

# Evaluate Model Performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Explain Model Predictions using SHAP
explainer = shap.TreeExplainer(xgb_model, X_train)
shap_values = explainer(X_test)  # SHAP values for test set

We will use the same test sample as before to enable a direct comparison of the results.

In [ ]:
# Select an example loan case (e.g., first sample in test set)
sample_index = 10  # Change this index if needed
shap_values_sample = shap_values[sample_index].values
sample_features = X_test.iloc[sample_index]

# Convert SHAP values into dictionary format for input
shap_dict = {feature: shap_values_sample[i] for i, feature in enumerate(sample_features.index)}

# Target label for the selected sample
pred_label = y_pred[sample_index]

print("\nSHAP Values for Sample Client:\n", shap_dict)

In [ ]:
shap.plots.bar(shap_values[sample_index])

In [ ]:
def format_shap_explanation(system_message, sample_features, shap_dict, pred_label, scaler):
    # Convert scaled numerical values back to original values
    original_values = scaler.inverse_transform(sample_features[numerical_columns].values.reshape(1, -1))
    original_feature_values = {feature: original_values[0][i] for i, feature in enumerate(numerical_columns)}

    # Sort SHAP values by absolute magnitude (most impactful features first)
    top_features = sorted(shap_dict.items(), key=lambda x: abs(x[1]), reverse=True)[:3]

    # Generate SHAP explanation text
    shap_text = "\n".join(
        [f"{feature}: SHAP value = {shap_value:.4f}, feature value = {original_feature_values[feature]:.2f}"
         for feature, shap_value in top_features]
    )

    print(shap_text)
    # Define loan delinquency status
    delinquency_status = "likely to be delinquent" if pred_label == 1 else "unlikely to be delinquent"
    print("prediected delinquency: ", delinquency_status)

    # Construct a revised prompt with explicit instructions and structured format
    user_prompt = f"""
The model predicts that the client is {delinquency_status}.

Here are the three most important features influencing the prediction:

{shap_text}

### Instructions:
- Analyze how each of three features contributes to the prediction.
- **Use correct feature names, not feature values.**
- **Strictly follow the structured response format.**
- **SHAP values must be interpreted correctly**:
  - **A positive SHAP value means the feature increases delinquency risk.**
  - **A negative SHAP value means the feature decreases delinquency risk.**


Now begin your structured analysis:
"""


    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [ ]:
system_message = system_message = """
You are a financial risk analyst. Your job is to analyze SHAP values and provide structured, fact-based explanations of the model's predictions.
"""

Finally, let's test the model and see how much it has improved compared to before.

In [ ]:
set_seed(48)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")


with torch.autocast("cuda", dtype=torch.bfloat16), torch.inference_mode():
  outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True,
    top_p=0.9,
    temperature=0.1,
    repetition_penalty=1,
    num_return_sequences=1,
    eos_token_id=tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    pad_token_id=tokenizer.pad_token_id,
    )

text_raw = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print("------------------------Fine-tuned model result---------------------------")
text = text_raw.split("assistant")[1].strip()
display(Markdown(text))

Looks great! We can see improvements in both readability and reasoning. You can compare these results with those from the previous lab, which used the original Llama, in the cell below.

In [ ]:
result_from_original_model =  """Feature Name: Credit Score Effect on Risk: A higher credit score reduces delinquency risk because it indicates a strong repayment history and financial responsibility. SHAP Impact: The SHAP value 0.6744 shows that including credit score for this client increases the probability of delinquency, meaning the model considers this a moderate indicator of financial reliability.

Feature Name: Number of Borrowers Effect on Risk: A higher number of borrowers increases delinquency risk because it may indicate financial strain and reduced ability to repay. SHAP Impact: The SHAP value 0.2054 shows that including number of borrowers for this client increases the probability of delinquency, meaning the model considers this a moderate indicator of financial strain.

Feature Name: Original Loan Term Effect on Risk: A longer original loan term increases delinquency risk because it may lead to higher monthly payments and increased financial burden. SHAP Impact: The SHAP value 0.0350 shows that including original loan term for this client increases the probability of delinquency, meaning the model considers this a weak indicator of financial reliability."""

print("------------------------Original model result---------------------------")
display(
    Markdown(result_from_original_model))